In [1]:
import import_ipynb

In [3]:
from AQI import avg_data_2017,avg_data_2018
import requests
import sys
import pandas as pd
from bs4 import BeautifulSoup
import os
import csv

In [4]:
def met_data(month, year):
    
    file_html = open('Data/Html_Data/{}/{}.html'.format(year,month), 'rb')
    plain_text = file_html.read()

    tempD = []
    finalD = []

    soup = BeautifulSoup(plain_text, "lxml")
    for table in soup.findAll('table', {'class': 'medias mensuales numspan'}):
        for tbody in table:
            for tr in tbody:
                a = tr.get_text()
                tempD.append(a)

    rows = len(tempD) / 15

    for times in range(round(rows)):
        newtempD = []
        for i in range(15):
            newtempD.append(tempD[0])
            tempD.pop(0)
        finalD.append(newtempD)

    length = len(finalD)

    finalD.pop(length - 1)
    finalD.pop(0)

    for a in range(len(finalD)):
        finalD[a].pop(6)
        finalD[a].pop(13)
        finalD[a].pop(12)
        finalD[a].pop(11)
        finalD[a].pop(10)
        finalD[a].pop(9)
        finalD[a].pop(0)

    return finalD

def data_combine(year, cs):
    for a in pd.read_csv('Data/Real-Data/real_' + str(year) + '.csv', chunksize=cs):
        df = pd.DataFrame(data=a)
        mylist = df.values.tolist()
    return mylist


if __name__ == "__main__":
    if not os.path.exists("Data/Real-Data"):
        os.makedirs("Data/Real-Data")
    for year in range(2017, 2019):
        final_data = []
        with open('Data/Real-Data/real_' + str(year) + '.csv', 'w') as csvfile:
            wr = csv.writer(csvfile, dialect='excel')
            wr.writerow(
                ['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM', 'PM 2.5'])
        for month in range(1, 13):
            temp = met_data(month, year)
            final_data = final_data + temp
            
        pm = getattr(sys.modules[__name__], 'avg_data_{}'.format(year))()

        if len(pm) == 364:
            pm.insert(364, '-')

        for i in range(len(final_data)-1):
            # final[i].insert(0, i + 1)
            final_data[i].insert(8, pm[i])

        with open('Data/Real-Data/real_' + str(year) + '.csv', 'a') as csvfile:
            wr = csv.writer(csvfile, dialect='excel')
            for row in final_data:
                flag = 0
                for elem in row:
                    if elem == "" or elem == "-":
                        flag = 1
                if flag != 1:
                    wr.writerow(row)
                    
    data_2017 = data_combine(2017, 600)
    data_2018 = data_combine(2018, 600)
    
     
    total=data_2017+data_2018
    
    with open('Data/Real-Data/Real_Combine_test.csv', 'w') as csvfile:
        wr = csv.writer(csvfile, dialect='excel')
        wr.writerow(
            ['T', 'TM', 'Tm', 'SLP', 'H', 'VV', 'V', 'VM', 'PM 2.5'])
        wr.writerows(total)

In [5]:
df_test = pd.read_csv('Data/Real-Data/Real_Combine_test.csv')
df_test

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,15.6,23.0,9.0,1016.4,87.0,1.3,2.4,11.1,284.795833
1,17.4,24.0,9.3,1018.9,81.0,1.3,2.2,9.4,219.720833
2,16.7,24.3,9.5,1018.0,82.0,1.0,4.1,13.0,182.187500
3,14.7,24.4,13.5,1012.2,96.0,1.0,10.0,22.2,154.037500
4,15.4,20.0,10.2,1015.5,84.0,1.1,2.2,7.6,223.208333
...,...,...,...,...,...,...,...,...,...
354,11.3,22.3,4.7,1017.7,82.0,1.1,3.0,3.5,310.250000
355,12.3,19.4,6.2,1016.8,80.0,1.3,6.7,25.9,204.375000
356,11.1,20.6,3.6,1015.7,76.0,1.3,3.0,9.4,223.250000
357,11.6,21.4,3.4,1017.1,76.0,1.1,2.4,5.4,248.958333


T         0
TM        0
Tm        0
SLP       0
H         0
VV        0
V         0
VM        0
PM 2.5    1
dtype: int64